In [ ]:
import pandas as pd
import json

In [ ]:
def get_financial_multiplier(api_code, default_multiplier=1000):
    """
    Returns the multiplier for a given API code.
    
    Args:
        api_code: The financial API code (e.g., 'SI', 'ANT', 'EKA')
        default_multiplier: Multiplier for monetary values (default: 1000)
    
    Returns:
        int: Multiplier value (1 for no change, default_multiplier for monetary values)
    """
    
    # API codes that should be multiplied (monetary values in thousands)
    MONETARY_CODES = {
        # Revenue Items
        'ADI', 'SDI', 'SI', 'IPA', 'IPI', 'IPT', 'LI', 'costs', 'bruttofort',
        
        # Expense Items  
        'ADK', 'SDK', 'LTP', 'PNKO', 'FOU', 'VF', 'PERSOMK', 'FU', 'loner_ovriga',
        'loner_styrelse_vd', 'summa_rorelsekostnader',
        
        # Asset Items
        'SED', 'SAM', 'SOM', 'SV', 'SVA', 'SF', 'KBP', 'EB', 'MI', 'MOA', 'AOM',
        'ANF', 'ANFF', 'AFA', 'SFA', 'SIA', 'AIM', 'GLA', 'KPL', 'SRF', 'AAM',
        'AID', 'IFS', 'ITS', 'IAA', 'AIV', 'SIV', 'IVEI', 'AFM', 'AMF', 'balance',
        'IMMA', 'DRM', 'other_property_plant_and_equipment', 'MAI', 'noncurrent_investments',
        'KAPI', 'other_noncurrent_financial_assets', 'FAA', 'anlagsaktiviteter',
        'VL', 'TFST', 'THNP', 'ATG', 'VP', 'likvider', 'OMAI', 'summa_finansiella_anltillg',
        'other_intangible_assets',
        
        # Liability & Equity Items
        'SEK', 'AK', 'SG', 'SKG', 'SLG', 'LG', 'PG', 'AEK', 'AEKK', 'IRK', 'SIK',
        'SOK', 'AKG', 'AKR', 'ALG', 'ALK', 'ANLG', 'GTD', 'GTI', 'MAS', 'SKGKI',
        'KL', 'KVL', 'KSL', 'LFK', 'LTK', 'OBL', 'OBLG', 'PST', 'US', 'USF',
        'EFMA', 'USKAT', 'hensettelser', 'long_term_mortgage_debt', 'long_term_debt_to_banks',
        'LGNP', 'other_long_term_debt', 'ANSVL', 'KGNP', 'short_term_mortgage_debt',
        'short_term_debt_to_banks', 'VK', 'AG', 'PASSI', 'summa_langfristiga_skulder',
        
        # Financial Items
        'AFI', 'FI', 'FK', 'AFK', 'AFKK', 'ARI', 'ARK', 'RFF', 'RTK', 'NF', 'NFA',
        'NFO', 'FPN', 'other_net_financial_income',
        
        # Results & Special Items
        'DR', 'OR', 'ORS', 'AARS', 'AARK', 'PR', 'EI', 'EK', 'NE', 'EXP', 'SKATAAR',
        'KB', 'MU', 'FSUB', 'UTB', 'SUB', 'SUBE', 'SUBT', 'AVUB', 'resultat_e_avskrivningar',
        'resultat_e_finansnetto',
        
        # Provisions & Adjustments
        'SAP', 'AVS', 'AFV', 'FFUG', 'OFUG', 'PF', 'PSM', 'SOVE', 'OAE', 'OAEE',
        'OTFF', 'TPF', 'NVD', 'ORA', 'VAFI', 'VMF', 'VMFO', 'VØKF', 'VØKI',
        
        # Cash Flow Items
        'KON1', 'KON2', 'KON3', 'KON4', 'KON5', 'KON6', 'KON8', 'KON9', 'KON10',
        'UBNKA', 'KON11', 'LIMKA', 'KON12', 'KBPS',
        
        # Working Capital & Inventory
        'ARKA', 'AV', 'BE', 'BEE', 'BEEA', 'BET', 'EBEF', 'VBI', 'OOF', 'OOF3',
        
        # Taxes & Fees
        'BS', 'SKO', 'SKR', 'OFA', 'short_term_tax_payables', 'SLF', 'REVAN', 'REVHO',
        
        # Other Special Items
        'EGA', 'MAK', 'MAO', 'AKSB', 'FONE', 'LEDPS', 'LL', 'ANDLØ', 'OBLNM',
        'KSK', 'MIN', 'MINK', 'MA', 'KUF', 'ANIE', 'GTS', 'HUSLK', 'OER', 'OVR'
    }
    
    # Return multiplier if code is monetary, otherwise return 1
    return default_multiplier if api_code in MONETARY_CODES else 1

In [ ]:
companies = pd.read_csv("raw/Supabase Snippet Update Company Background Images.csv")
codes = pd.read_csv("raw/codes.csv")

In [ ]:
mapper = (codes.set_index("API_CODE").DESCRIPTION_ENGLISH).to_dict()

In [147]:
cleaned = []
for (_,company) in companies.iterrows():
    annual_accounts = json.loads(company.annual_accounts)
    items = []
    for annual_year in annual_accounts:
        for item in annual_year["accounts"]:
            if item["code"] not in mapper:
                #print("missing",item["code"],item["amount"])
                continue
            else:
                if item["amount"] is None:
                    continue
        
                key = mapper[item["code"]]
                value = float(item["amount"])
                if "%" in key:
                    value/=100
                    key = key.replace(" in %","")
                    #print(key,value)
                value *= get_financial_multiplier(item["code"])
                    
                #We should fix so its correct value here... somethimes percentage, soemtime SEK, sometimes MSEK
                items.append([annual_year["year"],key,value])
    company_dict = company[["company_id","name","organization_number","company_type","company_purpose","established_date","foundation_year","registered_for_payroll_tax","homepage","postal_address","visitor_address","nace_categories","location"]].to_dict()
    if len(items):
        company_dict["financiaL_data"] = json.loads(pd.DataFrame(items).pivot_table(index=0,columns=1,values=2).to_json(orient="index"))
    cleaned.append(company_dict)
    

In [150]:
pd.DataFrame(cleaned).to_parquet("hello_nest.parquet")

In [ ]:
? pd.pivot_table